In [1]:
%%capture
!pip install langgraph langsmith langchain_groq gradio

In [2]:
%%capture
!pip install langchain  langchain_communit

In [3]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq
from google.colab import userdata
import gradio as gr

In [4]:
groq_api_key=userdata.get('groq_api_key')
langsmith=userdata.get('langsmith_api_key')

In [5]:
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fe211eae380>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fe211eafb50>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

### Define the state structure

In [6]:
class State(TypedDict):
  messages:Annotated[list,add_messages]

### Define the chatbot function


In [7]:
def chatbot(state:State):
  return {"messages":llm.invoke(state['messages'])}

### Create the graph

In [8]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

### Run the chatbot in a console loop

In [9]:
while True:
  user_input=input("User: ")
  if user_input.lower() in ["quit","q","exit"]:
    print("Goodbye")
    break
  for event in graph.stream({'messages':("user",user_input)}):
    print(event.values())
    for value in event.values():
      print(value['messages'])
      print("Assistant:",value["messages"].content)

User: Hi, my name is Olfat
dict_values([{'messages': AIMessage(content="Hello Olfat, it's nice to meet you! \n\nWhat can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 17, 'total_tokens': 46, 'completion_time': 0.052727273, 'prompt_time': 8.2649e-05, 'queue_time': 0.021223091, 'total_time': 0.052809922}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a0b8d1fd-f719-4694-bfce-13a09a8eb06b-0', usage_metadata={'input_tokens': 17, 'output_tokens': 29, 'total_tokens': 46})}])
content="Hello Olfat, it's nice to meet you! \n\nWhat can I do for you today? 😊  \n\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 17, 'total_tokens': 46, 'completion_time': 0.052727273, 'prompt_time': 8.2649e-05, 'queue_time': 0.021223091, 'total_time': 0.052809922}, 'model_name': 'Gemma2-9b-It', 'system_fingerpr


## **Gradio UI**

In [10]:

def gradio_chatbot(user_input, history=[]):

    state = {"messages": [("user", user_input)]}

    for event in graph.stream(state):
        for value in event.values():
            response = value["messages"].content
            history.append((user_input, response))
            return history


with gr.Blocks(css="""
        .gradio-container {background-color: #2c2f33; color: white;}
        .chatbot {background-color: #23272a; color: white;}
        .input-container {background-color: #40444b;}
        button {background-color: #7289da; color: white;}
        button:hover {background-color: #5b6eae;}
    """) as interface:
    gr.Markdown("#  Chatbot with Integrated LLM and State Graph", elem_id="title")
    chatbot_ui = gr.Chatbot(label="Chatbot", elem_classes=["chatbot"])
    message = gr.Textbox(show_label=False, placeholder="Type your message here...", lines=1, elem_classes=["input-container"])
    send_button = gr.Button("Send")
    send_button.click(fn=gradio_chatbot, inputs=[message, chatbot_ui], outputs=chatbot_ui)

interface.launch()


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:249: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://24e627b8124389ec40.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
